# Tutorial 3 - LUPA

The main goal of this tutorial is to demonstrate how to set up a WEC design problem with a more complex and realistic setup. We use the [Lab Upgrade Point Absorber (LUPA)](https://pmec-osu.github.io/LUPA/) device, an open source two-body heaving point absorber under development by Oregon State University. A deep dive video demonstration of the LUPA device and its features can be viewed [here](https://www.youtube.com/watch?v=gCcAu7H9lQI). We will numerically replicate LUPA testing in the Large Wave Flume at the [O.H. Hinsdale Wave Research Laboratory](https://engineering.oregonstate.edu/wave-lab) in order to provide further design optimization for the WEC device concept. This tutorial builds on the previous ones and introduces: 
- a WEC comprised of multiple bodies
- setting up a WEC with multiple degrees of freedom (DOF) using generalized modes
- more complex PTO kinematics that depend on more than one WEC DOF 
- realistic constraints including generator maximum and continuous torque, and maximum rotational speed
- irregular waves
- mooring system dynamics

A secondary goal is for this notebook to serve as a tool for those who are planning to run experiments with the LUPA device, to inform their experiment design.  

As with previous tutorials, this tutorial consists of two parts, with the second section building upon the first.

1. [Optimal control of a two-body WEC](#1.-Optimal-control-of-a-two-body-WEC)
2. [Control co-design of the PTO sprocket sizing for maximum electrical power](#2.-Control-co-design-of-the-PTO-sprocket-sizing-for-maximum-electrical-power)

<p><img src="rendering.png" alt="Digital rendering of the LUPA" width="310"> <img src="flume.jpg" alt="Picture of the LUPA floating in the flume in calm water. Both bodies and mooring are visible." width="500"> <img src="flume_dry.jpg" alt="Picture of the LUPA in the dry flume. Both bodies and mooring are visible. A ladder and person next to it gives context of the scale." width="500"></p>

In [ ]:
import gmsh, pygmsh
import capytaine as cpy
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import wecopttool as wot

## 1. Optimal control of a two-body WEC

### WEC object
The creation of the `WEC` object is fundamentally identical to previous tutorials, where we use meshes of the WEC to create Capytaine `FloatingBody` objects, run BEM using Capytaine, and use the `WEC.from_bem()` method to create the `WEC` object. The key here is that the LUPA is a two-body device (consisting of a float and a spar), which move independently in heave but in unison for all other degrees of freedom. To model this in WecOptTool, we can create a `FloatingBody` object for each body separately with a heave DOF, combine them into a single object afterwards, and be sure the combined mass and inertia properties are properly set.

We will analyze the device in its four planar degrees of freedom: the heave of the buoy, heave of the spar, combined device surge, and combined device pitch, for a total of 4 degrees of freedom. Here we are using the generalized modes approach. Alternatively we could include all 3 planar DOF for each body separately and add two constraints for the pitch and surge to be equal for both bodies. 

### Geometry
The mass properties of the LUPA have been provided from measurements of the physical device by Oregon State University, as follows:

In [ ]:
# provided by OSU
float_mass_properties = {
    'mass': 248.721,
    'CG': [0.01, 0, 0.06],
    'MOI': [66.1686, 65.3344, 17.16],
}

spar_mass_properties = {
    'mass': 175.536,
    'CG': [0, 0, -1.3],
    'MOI': [253.6344, 250.4558, 12.746],
}

#### Mesh creation of the float

Here we create the mesh based on the dimensions provided by Oregon State University using *pygmsh* as in the other tutorials.

In [ ]:
# mesh
mesh_size_factor = 0.3
# TODO: UPDATE once Courtney sends float drawing
r1 = 1.0/2  # top radius
r2 = 0.4/2  # bottom radius
h1 = 0.5  
h2 = 0.21
freeboard = 0.3
r3 = 0.10/2 + 0.05  # hole radius  # TODO

with pygmsh.occ.Geometry() as geom:
    gmsh.option.setNumber('Mesh.MeshSizeFactor', mesh_size_factor)
    cyl = geom.add_cylinder([0, 0, 0], [0, 0, -h1], r1)
    cone = geom.add_cone([0, 0, -h1], [0, 0, -h2], r1, r2)
    geom.translate(cyl, [0, 0, freeboard])
    geom.translate(cone, [0, 0, freeboard])
    tmp = geom.boolean_union([cyl, cone])
    bar = geom.add_cylinder([0, 0, 10], [0,0,-20], r3)
    geom.boolean_difference(tmp, bar)
    mesh_float = geom.generate_mesh()

Again, we will only add the heave DOF for now. The surge and pitch will be added after we combine the two `FloatingBody` objects.

In [ ]:
# floating body
float_fb = cpy.FloatingBody.from_meshio(mesh_float, name='float')
float_fb.add_translation_dof(name='Heave')

We can now visualize the mesh. 

In [ ]:
# show
float_fb.show_matplotlib()
# float_fb.show()  # interactive, close pop-up image before being able to continue running the notebook

#### Mesh creation of the spar

We now create the spar mesh in the same way as for the float, with the dimensions given by Oregon State University.

<p><img src="spar.png" alt="spar dimensions schematic from OSU" width=700></p>

In [ ]:
# mesh
mesh_size_factor = 0.1
r1 = 0.45/2  # body
r2 = 0.45  # plate
r3 = 0.10/2  # bar
h1 = 1.20  
h2 = 0.01
h3a = 3.684 - 2.05
submergence = 2.05 - h1 - h2

with pygmsh.occ.Geometry() as geom:
    gmsh.option.setNumber('Mesh.MeshSizeFactor', mesh_size_factor)
    body = geom.add_cylinder([0, 0, 0], [0, 0, -h1], r1)
    geom.translate(body, [0, 0, -submergence])
    plate = geom.add_cylinder([0, 0, 0], [0, 0, -h2], r2)
    geom.translate(plate, [0, 0, -(submergence+h1)])
    bar = geom.add_cylinder([0, 0, h3a], [0, 0, -(h3a+submergence)], r3)
    geom.boolean_union([bar, body, plate])
    mesh_spar = geom.generate_mesh()

In [ ]:
# floating body
spar_fb = cpy.FloatingBody.from_meshio(mesh_spar, name='spar')
spar_fb.add_translation_dof(name='Heave')

In [ ]:
# show
fig = plt.figure()
xmin, xmax, ymin, ymax, zmin, zmax = spar_fb.mesh.axis_aligned_bbox
ax = fig.add_subplot(111, projection='3d')
spar_fb.show_matplotlib(ax=ax)
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
# spar_fb.show()  # interactive, close pop-up image before being able to continue running the notebook

### Combined `FloatingBody`

With both WEC bodies defined separately, we can now create a union of the bodies and define the properties for the overall LUPA device. At the equilibrium position the float is neutrally buoyant while the spar is positively buoyant and requires mooring pre-tension. The combined center of mass and moment of inertia can be found by using the given values weighted by the mass (via the parallel axis theorem for the moment of inertia). This is also when we can specify the surge and pitch degrees of freedom.

We are using the density of *fresh* water, $\rho = 1000 kg/m^3$, since we are modeling LUPA in a wave flume.

In [ ]:
# density of fresh water
rho = 1000

# floating body
lupa_fb = float_fb + spar_fb
lupa_fb.name = 'LUPA'

# mass properties float
mass_float = float_mass_properties['mass'] 
cm_float = np.array(float_mass_properties['CG'])
pitch_inertia_float = float_mass_properties['MOI'][1] 

# mass properties spar
mass_spar = spar_mass_properties['mass']  
cm_spar = np.array(spar_mass_properties['CG'])
pitch_inertia_spar = spar_mass_properties['MOI'][1] 

 # mass properties LUPA
lupa_fb.center_of_mass = (mass_float*cm_float + mass_spar*cm_spar) / (mass_float + mass_spar)
lupa_fb.rotation_center = lupa_fb.center_of_mass

# pitch moment of inertia of LUPA using the parallel axis theorem 
d_float = cm_float[2] - lupa_fb.center_of_mass[2]
d_spar = cm_spar[2] - lupa_fb.center_of_mass[2]
pitch_inertia = (
    pitch_inertia_float + mass_float*d_float**2 + 
    pitch_inertia_spar + mass_spar*d_spar**2
)
inertia = np.diag([mass_float, mass_spar, lupa_fb.disp_mass(), pitch_inertia])

# additional DOFs
lupa_fb.add_translation_dof(name='Surge')
lupa_fb.add_rotation_dof(name='Pitch')

We can now visualize the combined mesh.

In [ ]:
# show
fig = plt.figure()
xmin, xmax, ymin, ymax, zmin, zmax = lupa_fb.mesh.axis_aligned_bbox
ax = fig.add_subplot(111, projection='3d')
lupa_fb.show_matplotlib(ax=ax)
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
# lupa_fb.show()

### BEM
With the LUPA geometry and physical properties fully defined, we can now run Capytaine to calculate the hydrodynamic and hydrostatic coefficients of the device, as done in previous tutorials. Capytaine can handle generalized modes and will calculate the coefficients for our 4 degrees of freedom.

In [ ]:
# compute hydrodynamic coefficients
f1 = 0.01
nfreq = 50
freq = wot.frequency(f1, nfreq, False)
# bem_data = wot.run_bem(lupa_fb, freq)
bem_data = wot.read_netcdf('lupa_bem.nc')

# hydrostatics
_ = lupa_fb.compute_hydrostatics(rho=rho)
hydrostatic_stiffness = lupa_fb.hydrostatic_stiffness

In [ ]:
# plot coefficients
radiating_dofs = ["float__Heave", "spar__Heave", "Surge", "Pitch"]
influenced_dofs = ["float__Heave", "spar__Heave", "Surge", "Pitch"]

# plots

fig_am, ax_am = plt.subplots(len(radiating_dofs), len(influenced_dofs), tight_layout=True, sharex=True)
fig_rd, ax_rd = plt.subplots(len(radiating_dofs), len(influenced_dofs), tight_layout=True, sharex=True)
fig_ex, ax_ex = plt.subplots(len(influenced_dofs), 1, tight_layout=True, sharex=True)

# plot titles
fig_am.suptitle('Added Mass Coefficients', fontweight='bold')
fig_rd.suptitle('Radiation Damping Coefficients', fontweight='bold')
fig_ex.suptitle('Wave Excitation Coefficients', fontweight='bold')

# subplotting across 4DOF
sp_idx = 0
for i, rdof in enumerate(radiating_dofs):
    for j, idof in enumerate(influenced_dofs):
        sp_idx += 1

        if i == 0:
            np.abs(bem_data.diffraction_force.sel(influenced_dof=idof)).plot(ax=ax_ex[j], linestyle='dashed', label='Diffraction force')
            np.abs(bem_data.Froude_Krylov_force.sel(influenced_dof=idof)).plot(ax=ax_ex[j], linestyle='dashdot', label='Froude-Krylov force')
            ex_handles, ex_labels = ax_ex[j].get_legend_handles_labels()
            ax_ex[j].set_title(f'{idof}', fontsize=8.5)
            ax_ex[j].set_xlabel('')
            ax_ex[j].set_ylabel('')
        if j <= i:
            bem_data.added_mass.sel(radiating_dof=rdof, influenced_dof=idof).plot(ax=ax_am[i, j])
            bem_data.radiation_damping.sel(radiating_dof=rdof, influenced_dof=idof).plot(ax=ax_rd[i, j])
            if i == j:
                ax_am[i, j].set_title(f'{idof}', fontsize=10)
                ax_rd[i, j].set_title(f'{idof}', fontsize=10)
            else:
                ax_am[i, j].set_title('')
                ax_rd[i, j].set_title('')
            if j == 0:
                ax_am[i, j].set_ylabel(f'{rdof}', fontsize=10)
                ax_rd[i, j].set_ylabel(f'{rdof}', fontsize=10)
            else:
                ax_am[i, j].set_ylabel('')
                ax_rd[i, j].set_ylabel('')
            ax_am[i, j].set_xlabel('')
            ax_rd[i, j].set_xlabel('')
            
        else:
            fig_am.delaxes(ax_am[i, j])
            fig_rd.delaxes(ax_rd[i, j])
fig_am.text(0.5, 0., '$\omega$', ha='center')
fig_rd.text(0.5, 0., '$\omega$', ha='center')
fig_ex.text(0.5, -0.08, '$\omega$', ha='center')
fig_ex.legend(ex_handles, ex_labels, loc=(0.22, 0.05), ncol=2)

## PTO system

The PTO model is similar to the one developed in Tutorial 2 but using the values corresponding to the LUPA PTO. 
The main difference is that in the LUPA the gear ratio can be modified by changing an interchangeable sprocket. 

The PTO system consists of a [generator](https://akribis-systems.s3-us-west-2.amazonaws.com/pdfs/catalogs/adr-b.pdf), the [interchangeable sprocket](https://assets.gates.com/content/dam/gates/home/knowledge-center/resource-library/catalogs/old-pc_carbon_manual17595_2011.pdf), and two [idler pulleys](https://dpk3n3gg92jwt.cloudfront.net/domains/gates.pt/pdf/77234200.pdf) driven by a belt.

<p><img src="pto.png" alt="Schematic of the PTO system showing dimensions and parts." width=740> <img src="pto_rendering.png" alt="Digital rendering of the LUPA with PTO mechanism visible." width=458></p>

We start by defining all the manufacturer-specified components.

In [ ]:
conv_d = 0.0254  # in -> m
conv_m = 0.453592  # lb -> kg
conv_moi = 0.453592 * 0.3048**2  # lb*ft^2 -> kg*m^2
conv_s = 2*np.pi,  # rpm -> rad/s

# page 65-68 of manual
sprockets = {
    '8MX-32S-36': {
        'diameter': 3.208 * conv_d,
        'mass': 1.7 * conv_m,
        'MOI': 0.02 * conv_moi,
        'design': 'AF-1',
    },
    '8MX-33S-36': {
        'diameter': 3.308 * conv_d,
        'mass':  3.31* conv_m,
        'MOI': 0.022 * conv_moi,
        'design': 'AF',
    },
    '8MX-34S-36': {
        'diameter': 3.409 * conv_d,
        'mass': 1.8 * conv_m,
        'MOI': 0.026 * conv_moi,
        'design': 'AF-1',
    },
    '8MX-35S-36': {
        'diameter': 3.509 * conv_d,
        'mass': 3.51 * conv_m,
        'MOI': 0.029 * conv_moi,
        'design': 'AF',
    },
    '8MX-36S-36': {
        'diameter': 3.609 * conv_d,
        'mass': 2.1 * conv_m,
        'MOI': 0.032 * conv_moi,
        'design': 'AF-1',
    },
    '8MX-37S-36': {
        'diameter': 3.709 * conv_d,
        'mass': 3.78 * conv_m,
        'MOI': 0.039 * conv_moi,
        'design': 'AF',
    },
    '8MX-38S-36': {
        'diameter': 3.810 * conv_d,
        'mass': 2.4 * conv_m,
        'MOI': 0.04 * conv_moi,
        'design': 'AF-1',
    },
    '8MX-39S-36': {
        'diameter': 3.910 * conv_d,
        'mass': 3.91 * conv_m,
        'MOI': 0.048 * conv_moi,
        'design': 'AF',
    },
    '8MX-40S-36': {
        'diameter': 4.010 * conv_d,
        'mass': 2.5 * conv_m,
        'MOI': 0.049 * conv_moi,
        'design': 'AF-1',
    },
    '8MX-41S-36': {
        'diameter': 4.110 * conv_d,
        'mass': 4.11 * conv_m,
        'MOI': 0.057 * conv_moi,
        'design': 'AF',
    },
    '8MX-42S-36': {
        'diameter': 4.211 * conv_d,
        'mass': 2.8 * conv_m,
        'MOI': 0.061 * conv_moi,
        'design': 'AF-1',  
    },
    '8MX-45S-36': {
        'diameter': 4.511 * conv_d,
        'mass': 3.8 * conv_m,
        'MOI': 0.09 * conv_moi,
        'design': 'AF-1',
    },
    '8MX-48S-36': {
        'diameter': 4.812 * conv_d,
        'mass': 4.3 * conv_m,
        'MOI': 0.114 * conv_moi,
        'design': 'AF-1',
    },
    '8MX-50S-36': {
        'diameter': 5.013 * conv_d,
        'mass': 5.1 * conv_m,
        'MOI': 0.143 * conv_moi,
        'design': 'AF-1',
    },
    '8MX-53S-36': {
        'diameter': 5.314 * conv_d,
        'mass': 5.5 * conv_m,
        'MOI': 0.169 * conv_moi,
        'design': 'AF-1',
    },
    '8MX-56S-36': {
        'diameter': 5.614 * conv_d,
        'mass': 6.5 * conv_m,
        'MOI': 0.221 * conv_moi,
        'design': 'AF-1',
    },
    '8MX-60S-36': {
        'diameter': 6.015 * conv_d,
        'mass': 8.9 * conv_m,
        'MOI': 0.352 * conv_moi,
        'design': 'AF-1',
    },
    '8MX-63S-36': {
        'diameter': 6.316 * conv_d,
        'mass': 10.4 * conv_m,
        'MOI': 0.556 * conv_moi,
        'design': 'AF-1',
    },
    '8MX-67S-36': {
        'diameter': 6.717 * conv_d,
        'mass': 6.5 * conv_m,
        'MOI': 0.307 * conv_moi,
        'design': 'DF-1',
    },
    '8MX-71S-36': {
        'diameter': 7.118 * conv_d,
        'mass': 7.0 * conv_m,
        'MOI': 0.365 * conv_moi,
        'design': 'DF-1',
    },
    '8MX-75S-36': {
        'diameter': 7.519 * conv_d,
        'mass': 7.3 * conv_m,
        'MOI': 0.423 * conv_moi, 
        'design': 'DF-1',
    },
    '8MX-80S-36': {
        'diameter': 8.020 * conv_d,
        'mass': 17.9 * conv_m,
        'MOI': 1.202 * conv_moi,  
        'design': 'BF-1',
    },
}

# https://dpk3n3gg92jwt.cloudfront.net/domains/gates.pt/pdf/77234200.pdf
idler_pulley = {
    'model': 'Gates 4.25X2.00-IDL-FLAT',
    'diameter': 4.25 * conv_d, 
    'face_width': 2.00 * conv_d,
    'mass': 7.6 * conv_m,
    'max_rpm': 5840,
    'MOI': None,  # TODO
}

# generator. Note: Model ADR220-B175 data not listed online, but we have spec sheet available on request 
generator = {
    'torque_constant': 8.51,  # N*m/A
    'winding_resistance': 5.87,  # Ω 
    'winding_inductance' : 0.0536,  # H 
    'max_torque': 137.9,  # N*m
    'continuous_torque': 46,  # N*M
    'max_speed': 150 * conv_s,  # rad/s
}

In [ ]:
# PTO
def gear_ratio(pulley_radius):
    return 1/pulley_radius  # rad/m 

def pto_impedance(pulley_radius, omega=bem_data.omega.values):
    # ADR220-B175-S/P-J/K-3.0-RA-26B-P25-Z75

    drivetrain_inertia = 2.0 / 2  # Kg*m^2 # TODO: rotor inertia 1.8e-2 + inertia of other 3 pulleys 
    drivetrain_friction = 0.5  # N*m*s/rad   
    drivetrain_stiffness = 0.0  # N*m/rad 
    drivetrain_impedance = (1j*omega*drivetrain_inertia + 
                            drivetrain_friction + 
                            1/(1j*omega)*drivetrain_stiffness) 
    winding_impedance = winding_resistance + 1j*omega*winding_inductance
    pto_impedance_11 = -1* gear_ratio(pulley_radius)**2 * drivetrain_impedance
    off_diag = -1*np.ones(omega.shape) * (
        np.sqrt(3.0/2.0) * torque_constant * gear_ratio(pulley_radius) + 0j)
    pto_impedance_12 = off_diag 
    pto_impedance_21 = off_diag
    pto_impedance_22 = winding_impedance
    impedance = np.array([[pto_impedance_11, pto_impedance_12],
                          [pto_impedance_21, pto_impedance_22]])
    return impedance

name = ["PTO_Heave",]
kinematics = np.array([[1, -1, 0, 0],])
pto_ndof = 1
controller = None
loss = None
min_radius = 0.0815
max_radius = 0.2037
mid_radius = 0.1273
default_radii = [min_radius, mid_radius, max_radius] 
pto = wot.pto.PTO(pto_ndof, kinematics, controller, pto_impedance(mid_radius), loss, name)

## Constraints TODO

In [ ]:
# Constraints
constraints = None  # TODO
nsubsteps = 5

## Geometry/displacements
# minimum relative position between float and spar (soft end stop)
stroke_max = 0.5  # m
def const_stroke_pto(wec, x_wec, x_opt, waves): 
    pos = pto.position(wec, x_wec, x_opt, waves, nsubsteps)
    return pos - np.abs(stroke_max.flatten())

# maximum position of spar: (hit top beam?)
# minimum position of spar: (hit bottom?)

## GENERATOR
# peak torque (generator) 
def const_peak_torque_pto(wec, x_wec, x_opt, waves): 
    """Instantaneous torque must not exceed max torque Tmax - |T| >=0 
    """
    torque = pto.force(wec, x_wec, x_opt, waves, nsubsteps) / gear_ratio(mid_radius)
    return torque_peak - np.abs(torque.flatten())

# continuous torque (generator)
def const_torque_pto(wec, x_wec, x_opt, waves): 
    """RMS torque must not exceed max continous torque 
        Tmax_conti - Trms >=0 """
    torque = pto.force(wec, x_wec, x_opt, waves, nsubsteps) / gear_ratio(mid_radius)
    torque_rms = np.sqrt(np.mean(torque**2))
    return torque_continuous - np.abs(torque_rms.flatten())

# max speed (generator)
def const_speed_pto(wec, x_wec, x_opt, waves): 
    rot_vel = pto.velocity(wec, x_wec, x_opt, waves, nsubsteps) * gear_ratio(mid_radius)
    return rot_speed - np.abs(rot_vel.flatten())

## Mooring system TODO

In [ ]:
# mooring matrix
def k_mooring(fair_coords, anch_coords, pretension, k_ax, nlines):
    """Calculates the 7DOF effective stiffness matrix of a symmetric taut
    mooring system using an analytical solution.
    """

    theta = np.arctan(
        (fair_coords[2] - anch_coords[2])**2
      / np.sqrt(((fair_coords[0] - anch_coords[0])**2
               + (fair_coords[1] - anch_coords[1])**2)))
    linelen = np.sqrt((fair_coords[0] - anch_coords[0])**2
                + (fair_coords[1] - anch_coords[1])**2
                + (fair_coords[2] - anch_coords[2])**2)
    fair_r = np.sqrt(fair_coords[0]**2 + fair_coords[1]**2)
    fair_z = -fair_coords[2]
    k_hh = 0.5 * nlines * (
        pretension / linelen * (1 + np.sin(theta)**2)
        + k_ax * np.cos(theta)**2)
    k_rh = nlines * (
        pretension / (2*linelen) * (fair_z * (1 + np.sin(theta)**2)
        + fair_r * np.sin(theta) * np.cos(theta))
        + 0.5 * k_ax * (fair_z * np.cos(theta)**2
        - fair_r * np.sin(theta) * np.cos(theta)))
    k_vv = nlines * (pretension / linelen *
        np.cos(theta)**2 + k_ax * np.sin(theta)**2)
    k_rr = nlines * (
        pretension * (fair_z * np.sin(theta) + 0.5 * fair_r * np.cos(theta))
        + (0.5 * pretension / linelen * ((fair_r * np.cos(theta) + fair_z * np.sin(theta))**2
        + fair_z**2))
        + 0.5 * k_ax * (fair_z * np.cos(theta) - fair_r*np.sin(theta))**2)
    k_tt = nlines * (
        pretension * fair_r / linelen * (fair_r + linelen*np.cos(theta)))
    mat = np.zeros([7, 7])
    mat[1, 1] = k_vv
    mat[2, 2] = k_hh
    mat[3, 3] = k_hh
    mat[4, 4] = k_rr
    mat[5, 5] = k_rr
    mat[6, 6] = k_tt
    mat[2, 5] = -k_rh
    mat[5, 2] = -k_rh
    mat[4, 3] = k_rh
    mat[3, 4] = k_rh

    return mat


pretension = 285
init_fair_coords = np.array([[-0.19, -0.19, -0.228],
                             [-0.19,  0.19, -0.228],
                             [ 0.19, -0.19, -0.228],
                             [ 0.19,  0.19, -0.228]])
anch_coords = np.array([[-1.95, -1.6, -0.368],
                        [-1.95,  1.6, -0.368],
                        [ 1.95, -1.6, -0.368],
                        [ 1.95,  1.6, -0.368]])
line_ax_stiff = 963.

M = k_mooring(init_fair_coords[0, :], anch_coords[0, :], pretension,
              line_ax_stiff, init_fair_coords.shape[0])
ind_4dof = np.array([0, 1, 2, 5])
M_4dof = M[np.ix_(ind_4dof, ind_4dof)]

In [ ]:
M_4dof

In [ ]:
# Additional Forces

# mooring
M = xr.DataArray(M_4dof, coords=[bem_data.coords['influenced_dof'], bem_data.coords['radiating_dof']], dims=['influenced_dof', 'radiating_dof'])
moor = ((M + 0j).expand_dims({"omega": bem_data.omega}))
tmp = moor.isel(omega=0).copy(deep=True)
tmp['omega'] = tmp['omega'] * 0
moor = xr.concat([tmp, moor], dim='omega') 
moor = moor.transpose("radiating_dof", "influenced_dof", "omega")
moor = -1*moor  # TODO RHS of equation: -ma = Σf 
mooring_force = wot.force_from_rao_transfer_function(moor, True)

# TODO: pre-tension heave spar

f_add = {
    'PTO': pto.force_on_wec,
    'Mooring': mooring_force
}

friction = np.diag([0, 0, 0, 0])

In [ ]:
# WEC
wec = wot.WEC.from_bem(bem_data,
                       inertia_matrix=inertia,
                       hydrostatic_stiffness=hydrostatic_stiffness,
                       constraints=constraints,
                       friction=friction,
                       f_add=f_add
)

## Waves

In [ ]:
# regular (test/setup)
amplitude = 0.1  
wavefreq = 0.5
phase = 0
wavedir = 0
waves_reg = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)

# LWF (South)
cases = {
    'max_90': {'Hs': 0.21, 'Tp': 3.09}, 
    'max_annual': {'Hs': 0.13, 'Tp': 2.35},
    'max_occurrence': {'Hs': 0.07, 'Tp': 1.90},
    'min_10': {'Hs': 0.04, 'Tp': 1.48},  
}

case = cases['max_occurrence']  # CHANGE HERE! 
hs = case['Hs']
fp = 1/case['Tp']
spectrum = lambda f: wot.waves.jonswap_spectrum(f, fp, hs, gamma=3.3)
efth = wot.waves.omnidirectional_spectrum(f1, nfreq, spectrum, "JONSWAP")
waves = wot.waves.long_crested_wave(efth)

## Solve

In [ ]:
# Objective function
obj_fun = pto.average_power
nstate_opt = wec.ncomponents

# Solve
scale_x_wec = 1e1  
scale_x_opt = 1e-3  
scale_obj = 1e-2  

results = wec.solve(
    waves_reg,  # CHANGE HERE 
    obj_fun, 
    nstate_opt, 
    scale_x_wec=scale_x_wec,
    scale_x_opt=scale_x_opt,
    scale_obj=scale_obj,
)

print(f'Optimal average electrical power: {-results.fun} W')

# Post-process
nsubsteps = 5
wec_fdom, wec_tdom = wec.post_process(results, waves, nsubsteps)
pto_fdom, pto_tdom = pto.post_process(wec, results, waves, nsubsteps)

## Results

In [ ]:
wec_tdom.pos.sel(influenced_dof="DOF_0").plot()

In [ ]:
pto_tdom.pos.plot()

## 2. Control co-design of the PTO sprocket sizing for maximum electrical power